In [6]:
import logging
from pathlib import Path

import geopandas as gpd
import matplotlib.pyplot as plt
import momepy
import networkx as nx
import numpy as np
import pandas as pd
import xarray as xr
from dotenv import dotenv_values

from generator_drainage_units import GeneratorCulvertLocations
from generator_drainage_units.utils.create_graph import create_graph_from_edges

logging.basicConfig(level=logging.DEBUG)

In [ ]:
%load_ext autoreload
%autoreload 2

In [8]:
config = dotenv_values("..\\.env")

base_dir = Path(config["BASE_DIR"])
dir_basis_data = "0_basisdata"
dir_inter_results = "1_tussenresultaat"
dir_results = "2_resultaat"

case_name = "Leuvenumse_beek"

case_path = Path(base_dir, case_name)

In [ ]:
cg = GeneratorCulvertLocations(
    path=case_path,
    dir_basis_data=dir_basis_data,
    dir_inter_results=dir_inter_results,
    dir_results=dir_results,
    read_results=True,
)

In [ ]:
cg.get_shortest_path_from_overige_watergangen_to_hydroobjects();

In [ ]:
cg.generate_folium_map()